In [1]:
import pandas as pd

FILE_NO_AGRUPADOS = '../data/datos_limpios_filtrados.csv'
FILE_AGRUPADOS = '../data/Agrupacion total variabilidad/datos_limpios_agrupacion_total_variabilidad.csv'

datos_limpios_no_agrupados = pd.read_csv(FILE_NO_AGRUPADOS)
datos_limpios_agrupados = pd.read_csv(FILE_AGRUPADOS)

lead_time = 7

def stats(datos):
    
    global lead_time

    mean_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})

    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * lead_time, axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * lead_time, axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])
    data_completa['COEF_VAR'] = data_completa.apply(lambda x: x['STD'] / x['MEDIA'] if x['MEDIA'] != 0 else 0, axis=1)
    data_completa['SS'] = data_completa.apply(lambda x: 1.65 * x['STD'], axis=1)
    data_completa['STOCK_TEORICO'] = data_completa.apply(lambda x: x['MEDIA'] + x['SS'], axis=1)
    data_completa['PORCENTAJE_SS_DEL_TOTAL'] = data_completa.apply(lambda x: x['SS'] / x['STOCK_TEORICO'] if x['MEDIA'] != 0 else 0, axis=1)
    porcentaje_ss_del_total = data_completa['SS'].sum() / data_completa['STOCK_TEORICO'].sum()
    
    return data_completa, porcentaje_ss_del_total

In [2]:
datos_limpios_no_agrupados = datos_limpios_no_agrupados[(datos_limpios_no_agrupados['DESCR_CENDIST'] != 'CERVECERA') & 
                                                        (datos_limpios_no_agrupados['DESCR_CENDIST'] != 'MODELO')]
stats_no_agrupados, porcentaje_ss_no_agrupado = stats(datos_limpios_no_agrupados)
# stats_no_agrupados

In [3]:
datos_limpios_agrupados = datos_limpios_agrupados[(datos_limpios_agrupados['DESCR_CENDIST'] != 'CERVECERA') & 
                                                  (datos_limpios_agrupados['DESCR_CENDIST'] != 'MODELO')]
stats_agrupados, porcentaje_ss_agrupado = stats(datos_limpios_agrupados)
stats_agrupados = stats_agrupados.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
# stats_agrupados

In [4]:
demandas = stats_no_agrupados.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).sum().reset_index()
demandas = demandas[['ID_SKU_VENTA', 'DESCR_CENDIST', 'MEDIA']]
demandas_agregadas = demandas.groupby(by=['ID_SKU_VENTA']).sum().reset_index()[['ID_SKU_VENTA', 'MEDIA']]
# demandas

In [5]:
no_agr = stats_no_agrupados[['ID_SKU_VENTA', 'DESCR_CENDIST', 'STOCK_TEORICO']]
# no_agr

In [6]:
agr = stats_agrupados[['ID_SKU_VENTA', 'DESCR_CENDIST', 'STOCK_TEORICO']]
# agr

In [7]:
data_diferencias = no_agr.merge(agr, on=['ID_SKU_VENTA', 'DESCR_CENDIST'], how='outer')
data_diferencias = data_diferencias.rename(columns={'STOCK_TEORICO_x': 'STOCK_NO_AGRUPADO',
                                                    'STOCK_TEORICO_y': 'STOCK_AGRUPADO'})
data_diferencias = data_diferencias.fillna(0)
data_diferencias['DELTA'] = data_diferencias['STOCK_NO_AGRUPADO'] - data_diferencias['STOCK_AGRUPADO']
# data_diferencias

In [15]:
data_diferencias

,ID_SKU_VENTA,DESCR_CENDIST,STOCK_NO_AGRUPADO,STOCK_AGRUPADO,DELTA
0,515,ANTOFAGASTA,1.567188,2.276953,-0.709764
1,515,ARICA,0.274062,0.000000,0.274062
2,515,CALAMA,0.952106,0.000000,0.952106
3,515,CASTRO,0.616935,0.000000,0.616935
4,515,CHILLAN,1.120763,0.000000,1.120763
...,...,...,...,...,...
23094,870506,IQUIQUE,0.000000,0.004662,-0.004662
23095,870614,TALCAHUANO,0.000000,0.045746,-0.045746
23096,870871,VALDIVIA,0.000000,0.053383,-0.053383
23097,870872,ANTOFAGASTA,0.000000,0.009677,-0.009677


In [18]:
calculo_dias_venta = data_diferencias.merge(demandas, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])
calculo_dias_venta = calculo_dias_venta.rename(columns={'MEDIA': 'MEDIA_SEMANAL'})
calculo_dias_venta

,ID_SKU_VENTA,DESCR_CENDIST,STOCK_NO_AGRUPADO,STOCK_AGRUPADO,DELTA,MEDIA_SEMANAL
0,515,ANTOFAGASTA,1.567188,2.276953,-0.709764,0.400533
1,515,ARICA,0.274062,0.000000,0.274062,0.041857
2,515,CALAMA,0.952106,0.000000,0.952106,0.219591
3,515,CASTRO,0.616935,0.000000,0.616935,0.120733
4,515,CHILLAN,1.120763,0.000000,1.120763,0.276593
...,...,...,...,...,...,...
23007,875225,VALDIVIA,0.007065,0.036402,-0.029337,0.000280
23008,875359,SANTIAGO SUR,0.408838,0.408838,0.000000,0.008413
23009,875365,CURAUMA,0.582771,0.582771,0.000000,0.016827
23010,875365,SANTIAGO SUR,0.408838,0.408838,0.000000,0.008413


In [19]:
OUTPUT_FILE = '../data/Agrupacion total variabilidad/calculo_dias_venta_total_variabilidad.xlsx'
calculo_dias_venta.to_excel(OUTPUT_FILE)

# De aquí hacia abajo, ignorar.

In [8]:
diferencias_agregadas_sku = data_diferencias.groupby(by=['ID_SKU_VENTA']).sum().reset_index()
# diferencias_agregadas_sku

In [9]:
dias_venta_desagregados = demandas.merge(data_diferencias, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])
dias_venta_desagregados['DELTA_SEMANAS_VENTA'] = dias_venta_desagregados['DELTA'] / dias_venta_desagregados['MEDIA']
dias_venta_desagregados['DELTA_DIAS_VENTA'] = dias_venta_desagregados['DELTA_SEMANAS_VENTA'] * 7
dias_venta_desagregados = dias_venta_desagregados[['ID_SKU_VENTA', 'DESCR_CENDIST', 'MEDIA', 'DELTA_DIAS_VENTA']]
dias_venta_desagregados = dias_venta_desagregados.rename(columns={'MEDIA': 'MEDIA_SEMANAL'})
# dias_venta_desagregados

In [10]:
dias_venta_agregados = demandas.merge(diferencias_agregadas_sku, on=['ID_SKU_VENTA'])
dias_venta_agregados['DELTA_SEMANAS_VENTA'] = dias_venta_agregados['DELTA'] / dias_venta_agregados['MEDIA']
dias_venta_agregados['DELTA_DIAS_VENTA'] = dias_venta_agregados['DELTA_SEMANAS_VENTA'] * 7
dias_venta_agregados = dias_venta_agregados[['ID_SKU_VENTA', 'MEDIA', 'DELTA_DIAS_VENTA']]
dias_venta_agregados = dias_venta_agregados.rename(columns={'MEDIA': 'MEDIA_SEMANAL'})
# dias_venta_agregados

# Finalmente, almacenamos en los archivos correspondientes.

In [112]:
FILE_DESAGREGADOS = '../data/Agrupacion total variabilidad/dias_venta_desagregados_agrupacion_total_variabilidad.csv'
FILE_AGREGADOS = '../data/Agrupacion total variabilidad/dias_venta_agregados_agrupacion_total_variabilidad.csv'

dias_venta_desagregados.to_csv(FILE_DESAGREGADOS)
dias_venta_agregados.to_csv(FILE_AGREGADOS)